In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

In [ ]:
df = pd.read_csv("C:/Users/Yerko/Desktop/datasets/Owid.csv")
print(df.head)

In [ ]:

unique_countries = df['country'].unique()

pattern = re.compile(r"\s+\((EIA|Ember|Shift|EI|World)\)$")

filtered_countries = [country for country in unique_countries if not pattern.search(country)]

filtered_countries

In [ ]:

regions = {
    'Africa': [],
    'Asia': [],
    'Russia': ['Russia'],
    'North America': [],
    'South America': [],
    'East Europe': [],
    'West Europe': []
}

for country in filtered_countries:
    if country in ['Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cameroon', 'Cape Verde',
                   'Central African Republic', 'Chad', 'Comoros', 'Congo', 'Djibouti', 'Egypt', 'Equatorial Guinea',
                   'Eritrea', 'Eswatini', 'Ethiopia', 'Gabon', 'Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Ivory Coast',
                   'Kenya', 'Lesotho', 'Liberia', 'Libya', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius',
                   'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Sao Tome and Principe', 'Senegal',
                   'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan', 'Sudan', 'Tanzania', 'Togo',
                   'Tunisia', 'Uganda', 'Zambia', 'Zimbabwe']:
        regions['Africa'].append(country)
    elif country in ['Afghanistan', 'Armenia', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Bhutan', 'Brunei', 'Cambodia',
                     'China', 'Cyprus', 'Georgia', 'India', 'Indonesia', 'Iran', 'Iraq', 'Israel', 'Japan', 'Jordan',
                     'Kazakhstan', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Lebanon', 'Malaysia', 'Maldives', 'Mongolia',
                     'Myanmar', 'Nepal', 'North Korea', 'Oman', 'Pakistan', 'Palestine', 'Philippines', 'Qatar', 'Saudi Arabia',
                     'Singapore', 'South Korea', 'Sri Lanka', 'Syria', 'Taiwan', 'Tajikistan', 'Thailand', 'Timor-Leste',
                     'Turkey', 'Turkmenistan', 'United Arab Emirates', 'Uzbekistan', 'Vietnam', 'Yemen']:
        regions['Asia'].append(country)
    elif country in ['Canada', 'United States', 'Mexico', 'Greenland', 'Bermuda']:
        regions['North America'].append(country)
    elif country in ['Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 'Peru',
                     'Suriname', 'Uruguay', 'Venezuela']:
        regions['South America'].append(country)
    elif country in ['Albania', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Czechia', 'Estonia', 'Hungary',
                     'Kosovo', 'Latvia', 'Lithuania', 'Macedonia', 'Moldova', 'Montenegro', 'Poland', 'Romania', 
                     'Serbia', 'Slovakia', 'Slovenia', 'Ukraine', 'Belarus']:
        regions['East Europe'].append(country)
    elif country in ['Austria', 'Belgium', 'Cyprus', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Iceland',
                     'Ireland', 'Italy', 'Luxembourg', 'Malta', 'Netherlands', 'Norway', 'Portugal', 'Spain', 'Sweden',
                     'Switzerland', 'United Kingdom']:
        regions['West Europe'].append(country)

regions


In [ ]:

filtered_data = df[df['year'] >= 1985]

region_electricity = pd.DataFrame()

for region, countries in regions.items():
    region_data = filtered_data[filtered_data['country'].isin(countries)]
    
    aggregated_data = region_data.groupby('year')['nuclear_electricity'].sum().rename(region)
    
    if region_electricity.empty:
        region_electricity = aggregated_data.to_frame()
    else:
        region_electricity = region_electricity.join(aggregated_data, how='outer')

region_electricity.head()



In [ ]:


desired_order = ['Africa', 'Asia', 'Russia', 'North America', 'South America', 'East Europe', 'West Europe']
colors = ['#DAF7A6', '#3A80F5', '#87D7CD', '#BC2424', '#62350E', '#9581C9', '#FFD700']
labels = [
    'Nuclear Electricity from Africa',
    'Nuclear Electricity from Asia',
    'Nuclear Electricity from Russia',
    'Nuclear Electricity from North America',
    'Nuclear Electricity from South America',
    'Nuclear Electricity from East Europe',
    'Nuclear Electricity from West Europe'
]

plt.figure(figsize=(18, 10))
ax = plt.gca()
bottom = 0

for region, color, label in zip(desired_order, colors, labels):
    plt.bar(region_electricity.index, region_electricity[region], bottom=bottom, color=color, label=label)
    bottom += region_electricity[region]

plt.title('Nuclear Electricity Production by Region', fontsize=16, fontweight='bold')
plt.ylabel('Electricity Production (TWh)', fontsize=14)
plt.xlabel('Year', fontsize=14)
plt.xticks(rotation=45)
plt.xlim(left=1984)
plt.legend(loc='upper left')
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.5)
plt.tight_layout()

plt.show()


In [ ]:
data_S = region_electricity.iloc[:, :].loc[2022]

print("Nuclear Electricity Production (in TWh):")
print(data_S)

In [ ]:
forecast_path = "C:/Users/Yerko/Desktop/results/Updated_Nuclear_Electricity_Forecasts.csv"
forecast_df = pd.read_csv(forecast_path)

forecast_df.head()

In [ ]:
forecast_df.rename(columns={'Year': 'year'}, inplace=True)

combined_data = pd.concat([region_electricity, forecast_df.set_index('year')], axis=0)

plt.figure(figsize=(18, 10))
ax = plt.gca()
bottom = np.zeros_like(combined_data.index, dtype=float)  

for region, color, label in zip(desired_order, colors, labels):
    historical_part = combined_data.loc[combined_data.index <= 2022, region]  
    forecast_part = combined_data.loc[combined_data.index >= 2023, region]    

    plt.bar(historical_part.index, historical_part, bottom=bottom[combined_data.index <= 2022], color=color, label=label)
    plt.bar(forecast_part.index, forecast_part, bottom=bottom[combined_data.index >= 2023], color=color, alpha=0.5)

    bottom[combined_data.index <= 2022] += historical_part.values
    bottom[combined_data.index >= 2023] += forecast_part.values

plt.axvline(x=2022.5, color='black', linestyle='--', linewidth=1, label='Start of Forecast Data')

plt.title('Nuclear Electricity Production by Region (Historical & Forecast)', fontsize=16, fontweight='bold')
plt.ylabel('Electricity Production (TWh)', fontsize=14)
plt.xlabel('Year', fontsize=14)
plt.xticks(rotation=45)
plt.xlim(left=1984)
plt.legend(loc='upper left')
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.5)
plt.tight_layout()

plt.show()

In [ ]:
data_F = combined_data.loc[2030] if 2030 in combined_data.index else "No data available for 2030"

data_F
